In [1]:
import numpy as np

### Initialization part

In [2]:
def init_layers(batch_size,layer_sizes):
        hidden_layers = [np.empty((batch_size,layer_size)) for layer_size in layer_sizes]
        return hidden_layers

In [ ]:
def init_weights(layer_sizes):
        weights = list()
        for i in range(layer_sizes.shape[0]-1):
            weights.append(np.random.uniform(-1,1,size=[layer_sizes[i],layer_sizes[i+1]]))
        weights = asarray(self.weights)
        return weights

### Feed forward part


In [ ]:
def sigmoid(x):
    return 1./(1.+np.exp(-x))

In [ ]:
def softmax(x):
    exponent = np.exp(x) # only compute the exponent once
    return exponent/exponent.sum(axis=1,keepdims=True)

In [ ]:
def feed_forward(batch,hidden_layers,weights):
    h_l = batch
    hidden_layers[0] = h_l
    for i,weight in enumerate(weights):
        h_l = sigmoid(h_l.dot(weight))
        hidden_layers[i+1]=h_l
    output = softmax(hidden_layers[-1])
    return output, hidden_layers

### Backpropagarion part

In [ ]:
def sigmoid_derivative(sigmoid_out):
    """
    Calculate derivative of sigmoid activation based on sigmoid output.

    Parameters
    ----------
    sigmoid_out : array_like
        Output values processed by a sigmoid function.
    
    Returns
    -------
    sigmoid_prime(h) : array_like
        Derivative of sigmoid, based on value of sigmoid.
    """
    return sigmoid_out*(1-sigmoid_out)

In [ ]:
def back_prop(output,batch_y,hidden_layers,weights,batch_size,lr):
        delta_t = (output - batch_y)*sigmoid_derivative(hidden_layers[-1])
        for i in range(1,len(weights)+1):
            weights[-i]-=lr*(hidden_layers[-i-1].T.dot(delta_t))/batch_size
            delta_t = sigmoid_derivative(hidden_layers[-i-1])*(delta_t.dot(weights[-i].T))
        return weights